<a href="https://www.kaggle.com/code/siegfredlorellecmina/ai-avenue-2-introduction-to-computer-vision-nb-2?scriptVersionId=176311459" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# AI Avenue 2: Introduction to Computer Vision

## Fine-tuning Convolutional Neural Network (CNN)

### Steps:
1. Acquire Data
2. Preprocess Data
3. Build Fine-tuned Model
4. Train Fine-tuned Model
5. Predict using the Model

In [ ]:
# Import libraries used across the notebook
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sklearn
import matplotlib.pyplot as plt 
from PIL import Image
import numpy as np
import os
import shutil
import glob
import random
import itertools

In [ ]:
# Ensure a GPU is used by TensorFlow 
GPUs = tf.config.list_physical_devices('GPU')

try:
    tf.config.experimental.set_memory_growth(GPUs[0], True)
except:
    print("Dynamic memory allocation on GPU was not set.")
    
print(f"Num of GPUs: {len(GPUs)}")


### 1. Acquire Data
- Download dogs vs cats dataset named 'dogs-vs-cats.zip' from [this Google Drive](https://drive.google.com/drive/folders/14xi2BkY3D8UL77kWoPN30GJOcnVyfBWf)
- Upload the dogs vs cats dataset in Kaggle



<details>
    <summary>Detailed steps?</summary>
    
<!-- 1. #### Click 'Upload'
2. #### Click 'Create New Dataset'
3. #### Upload Dataset
4. #### Name Dataset then click 'Create'
5. #### Check if Dataset is uploaded
</details>


In [ ]:
# Check if dataset is downloaded

input_dataset_path = '/kaggle/input/dogs-vs-cats-for-gdsc/dogs-vs-cats'

files = os.listdir(input_dataset_path)
print(f"Num of files in input dataset: {len(files)}")


In [ ]:
# # View all files in the dataset
# files

### 2. Preprocess Data
- Separate data by label
-  Split the dataset into three (3) subsets
    - train set
    - valid set
    - test set
- Process the data to match the model's expected input

In [ ]:
# Clear/deletes all files in working directory
# Warning: Only run this to reset working directory from scratch (empty)

working_directory_path = "/kaggle/working/"

os.chdir(working_directory_path)
for file in os.listdir(working_directory_path):
    if os.path.isdir(file):
        shutil.rmtree(file)
    else:
        os.remove(file)
        
print(os.listdir(working_directory_path))

In [ ]:
# Copy dataset (from input) to working kaggle directory

input_dataset_path = "/kaggle/input/dogs-vs-cats-for-gdsc/dogs-vs-cats"
working_dir_path = "/kaggle/working/"

input_files = os.listdir(input_dataset_path)

src = input_dataset_path
dest = os.path.join(working_dir_path,"data")

if not os.path.exists(dest) or len(os.listdir(dest)) != len(input_files):
    shutil.copytree(src, dest, dirs_exist_ok=True)


files = os.listdir(dest)
print(f"Num of files in working directory: {len(files)}")


In [ ]:
# Create train, valid, test directories

# Move to the dataset in working directory
dataset_path = "/kaggle/working/data"
print(os.getcwd())
os.chdir(dataset_path)
print(os.getcwd())

# Create train, valid, test
train_dog_path = './train/dog'
if not os.path.exists(train_dog_path):
    os.makedirs(train_dog_path)
valid_dog_path = "./valid/dog"
if not os.path.exists(valid_dog_path):
    os.makedirs(valid_dog_path)
test_dog_path = "./test/dog"
if not os.path.exists(test_dog_path):
    os.makedirs(test_dog_path)
train_cat_path = './train/cat'
if not os.path.exists(train_cat_path):
    os.makedirs(train_cat_path)
valid_cat_path = "./valid/cat"
if not os.path.exists(valid_cat_path):
    os.makedirs(valid_cat_path)
test_cat_path = "./test/cat"
if not os.path.exists(test_cat_path):
    os.makedirs(test_cat_path)

# Check if directories are made by getting all subdirectories in working directory
dirs_in_working_dir = [file for file in os.listdir(os.getcwd()) if os.path.isdir(file)]
dirs_in_working_dir

In [ ]:
# Split dataset by labels
# & split into three (3) subsets

# Move to the dataset in working directory
dataset_path = "/kaggle/working/data"
print(os.getcwd())
os.chdir(dataset_path)
print(os.getcwd())



# Assign a ratio on how much of the dataset will be part of each subset 
subsets_ratio = {
    "train" : 0.7,
    "valid" : 0.2,
    "test" : 0.1,
}
# Ensure that the ratio totals to 1
subsets_ratio_sum = round(sum(subsets_ratio.values()), 1)
assert subsets_ratio_sum == 1, f"Subsets_ratio ({subsets_ratio_sum}) must be equal to 1" 

# Get the total count of filenames starting with dog or cat
# NOTE: Wildcard (*) selects all filenames starting with dog or cat
labels_total = {
    "dog": len(glob.glob("dog*")),
    "cat": len(glob.glob("cat*")),
}
    

# Split the dataset by labels and subset 
for subset, ratio in subsets_ratio.items():
    for label, total in labels_total.items():
        
        # Get a random sample of the current label by the ratio of the current subset
        num_of_files_to_move = int(ratio * total)
        files_to_move = random.sample(glob.glob(f"{label}*"), num_of_files_to_move)
        # Move the sampled files to its appropriate directory
        for file in files_to_move:
            src = file
            dest = os.path.join(subset, label)
            shutil.move(src, dest)

# Check if dataset was split correctly
for subset in subsets_ratio.keys():
    for label, total in labels_total.items():
        num_of_files = len(os.listdir(os.path.join(subset, label)))
        ratio = num_of_files / total if total else 'N/A'
        print(f"Num of files in {subset}/{label}: {num_of_files} files (ratio: {ratio})")

In [ ]:
# # View each directory manually
# os.listdir("train/dog")
# os.listdir("train/cat")
# os.listdir("valid/dog")
# os.listdir("valid/cat")
# os.listdir("test/dog")
# os.listdir("test/cat")

In [ ]:
# Process data to match the model's expected input

# Move to the dataset in working directory
dataset_path = "/kaggle/working/data"
print(os.getcwd())
os.chdir(dataset_path)
print(os.getcwd())

train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "valid")
test_path = os.path.join(dataset_path, "test")

target_size = (224, 224)
classes = ["cat","dog"]
batch_size = 64


data_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

train_batches = data_generator.flow_from_directory(
    directory=train_path,
    target_size=target_size,
    classes=classes,
    batch_size=batch_size,
)
valid_batches = data_generator.flow_from_directory(
    directory=valid_path,
    target_size=target_size,
    classes=classes,
    batch_size=batch_size,
)
test_batches = data_generator.flow_from_directory(
    directory=test_path,
    target_size=target_size,
    classes=classes,
    batch_size=batch_size,
    shuffle=False,
)

In [ ]:
# Visualize a batch of a preprocessed images
def plotImages(imgs_arr):
    fig, cell = plt.subplots(1, 10, figsize=(15, 15))
#     axes = axes.flatten()
    for img, cell in zip(imgs_arr, cell):
#         print(cell)
        cell.imshow(img)
        cell.axis("off")
#         cell.


In [ ]:
# Plot preprocessed images from train batch
imgs, labels = next(train_batches)
plotImages(imgs)
[print(f"{i}: {label}") for i, label in enumerate(labels, 1) if i <= 10]


# Plot random unpreprocessed images from train subset
images_paths = glob.glob(f"{dataset_path}/train/*/*")  
random_imgs = random.sample(images_paths, 10)

imgs_arr = []
for img in random_imgs:
    img = Image.open(img)
    img_arr = np.asarray(img)
    imgs_arr.append(img_arr)
plotImages(imgs_arr)

### 3. Build and Fine-tuning CNN
- Download/Grab Existing Model
- Create new model using the downloaded model's layer
- Adjust number of trainable parameters 

In [ ]:
# Download/grab existing model
vgg16_model = tf.keras.applications.vgg16.VGG16()

In [ ]:
# # Check vgg16 layers
# vgg16_model.summary()

In [ ]:
# Create new model using the downloaded model's layer
inputs = vgg16_model.inputs
x = vgg16_model.layers[-2].output
outputs = tf.keras.layers.Dense(
    units=2,
    activation="softmax",
) (x)

model = tf.keras.Model(
    inputs=inputs,
    outputs=outputs,
)

In [ ]:
# # Check new model's layers
# model.summary()

#### Adjust number of trainable parameters based on:



In [ ]:
# Adjust number of non-trainable parameters (to freeze)
num_of_layers_not_to_freeze = 1

for layer in model.layers[: -num_of_layers_not_to_freeze]:
    layer.trainable = False
    
for layer in model.layers[-num_of_layers_not_to_freeze :]:
    layer.trainable = True

In [ ]:
# model.summary()

### 4. Train Fine-tuned CNN
- Prepare model for training
- Train the model
- Plot train and valid accuracy

In [ ]:
# Prepare the model for training
learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = "categorical_crossentropy"
metrics = ["accuracy"]

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics,
)

In [ ]:
#  Train the model
history = model.fit(
    x=train_batches,
#     steps_per_epoch=train_batches.n//batch_size,
    validation_data=test_batches,
#     validation_steps=valid_batches.n//batch_size,
    epochs=5,
    verbose=1,
)

In [ ]:
# Plot train and valid accuracy
# TODO

### 5. Predict using the Model
- predict images from test batches
- visualize results through confusion matrix
- compute numerical results

In [ ]:
# Predict images from test batches
predictions = model.predict(
    x=test_batches,
    verbose=1,
)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting 'normalize=True'.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
        
    print(cm)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape [0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# # Check if test batches labels are still unshuffled
# for label in test_batches.classes:
#     print(label)

In [ ]:
# Create a confusion matrix to visualize results
confusion_matrix = sklearn.metrics.confusion_matrix(
    y_true=test_batches.classes,
    y_pred=np.argmax(predictions, axis=-1),
)

# Plot the confusion matrix
classes = ["Cat", "Dog"]
plot_confusion_matrix(
    cm=confusion_matrix, 
    classes=classes,
    cmap=plt.cm.YlGn,
)

In [ ]:
# Numerical Results
accuracy = sklearn.metrics.accuracy_score(
    test_batches.classes,
    np.argmax(predictions, axis=-1)
)

f"accuracy: {accuracy * 100:.2f}%"

In [ ]:
# Check each prediction results

    
predicted_labels = ["Cat" if np.argmax(prediction, axis=-1) == 0 else "Dog" for prediction in predictions]
true_labels = ["Cat" if label == 0 else "Dog" for label in test_batches.classes]

print(f"True : Predicted")
for i in range(len(predicted_labels)):
    print(f"{true_labels[i]} : {predicted_labels[i]} {'✅' if predicted_labels[i] == true_labels[i] else '❌'}")


### Additional:
- Save Model